In [1]:
import sys; sys.path.extend([".."])

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from wigner_time import timeline as tl
from wigner_time import display as dp
from wigner_time import adwin as ad
import wigner_time, pickle

import experimentDemo as ex
import diagnosticsDemo as di

%matplotlib qt

preparing sample and interweaving the absorption imaging anchored to the end of the optical pumping stage

In [2]:
timeline=di.imaging_absorption(5e-3,300e-6,context="AI",
                               origin="OP", # this says that the time of absorption imaging is relative to the anchor of context "OP", that is, the end of the optical pumping stage
                               timeline=di.prepareSample(stage=ex.Stage.OP,MOT_duration=1,initFunction=di.init,finishFunction=di.finish)
                              )

In [3]:
dp.display(tl.expand(timeline,
                     # prepareSampleStageSelect(stage=Stage.OP,MOT_duration=1),
                     time_resolution=25e-6))

(<Figure size 750x750 with 5 Axes>,
 array([<Axes: ylabel='Voltage [V]'>, <Axes: ylabel='Current [A]'>,
        <Axes: ylabel='Frequency [MHz]'>,
        <Axes: ylabel='Digital channels'>], dtype=object))

`prepare_atoms` is equivalent to stacking the different stages, like this (this time until the end of magnetic trapping):

In [4]:
timeline=tl.stack(
    di.init(),
    ex.MOT(),
    ex.MOT_detunedGrowth(),
    ex.molasses(),
    ex.OP(),
    ex.magneticTrapping(),
    ex.pull_coils(50e-3,-4.1,-4.7,-0.6,-0.6),
    ex.finish()
)

timeline=di.imaging_absorption(2.5e-3,300e-6,origin="magneticTrapping",context="AI",timeline=timeline)

`timeline` is simply a `pandas.DataFrame`:

In [ ]:
# pd.options.display.max_rows = 10_000
timeline

Notice that the above `timeline` contains the ramps in “compressed” form, that is, simply as functionals.

Ramps can be expanded with the following function:

In [6]:
tl.expand(timeline,time_resolution=5e-6)

,time,variable,value,context
0,-0.000001,lockbox_MOT__MHz,0.000000,ADwin_LowInit
1,-0.000001,coil_compensationX__A,0.250000,ADwin_LowInit
2,-0.000001,coil_compensationY__A,1.500000,ADwin_LowInit
3,-0.000001,coil_MOTlowerPlus__A,0.100000,ADwin_LowInit
4,-0.000001,coil_MOTupperPlus__A,-0.100000,ADwin_LowInit
...,...,...,...,...
59136,16.118160,coil_MOTupperPlus__A,-0.100036,finalRamps
59137,16.118165,coil_MOTupperPlus__A,-0.100027,finalRamps
59138,16.118170,coil_MOTupperPlus__A,-0.100018,finalRamps
59139,16.118175,coil_MOTupperPlus__A,-0.100009,finalRamps


In [7]:
#dp.display(timeline)
dp.display(tl.expand(timeline,time_resolution=5e-6))

(<Figure size 750x750 with 5 Axes>,
 array([<Axes: ylabel='Voltage [V]'>, <Axes: ylabel='Current [A]'>,
        <Axes: ylabel='Frequency [MHz]'>,
        <Axes: ylabel='Digital channels'>], dtype=object))

This would be the way to create the data that can be transferred to ADwin:

In [ ]:
ad.core.to_data(timeline,ex.connections,ex.devices)